In [8]:
import numpy as np
import functions as fn
from circuit_obj import Circuit
from tqdm import tqdm
from itertools import product


In [9]:
# Define your parameter arrays
N = 16
T = 10
masks_dict = fn.load_mask_memory(N, 2)

In [10]:
def initial_state(N, n):
    """
    Generate a random state vector on N qubits with spin up at site n
    """
    up = np.array([0, 1])
    if n == 0:
        rnd_part = np.random.rand(2**(N - 1)) + 1j * np.random.rand(2**(N - 1))
        state = np.kron(up, rnd_part)
        state /= np.linalg.norm(state)
        return state
    elif n == N - 1:
        rnd_part = np.random.rand(2**(N - 1)) + 1j * np.random.rand(2**(N - 1))
        state = np.kron(rnd_part, up)
        state /= np.linalg.norm(state)
        return state
    part1 = np.random.rand(2**(n)) + 1j * np.random.rand(2**(n))
    part2 = np.random.rand(2**(N - n - 1)) + 1j * np.random.rand(2**(N - n - 1))
    state = np.kron(part1, np.kron(up, part2))
    state /= np.linalg.norm(state)
    return state

# sanity check :
# [np.round(fn.get_magnetization(initial_state(N, idx), N)[idx]) for idx in range(N)]

st = initial_state(N, 0)

In [11]:
import json

with open('random_archive/angles_N16.json', 'r', encoding='utf-8') as f:
    loaded_dict = json.load(f)
Jz, θ1, Jx, θ2,θ3,θ4 = loaded_dict[0][0].values()
Jz, θ1, Jx, θ2,θ3,θ4

(-2.6994877483691653,
 0.2407780253979941,
 -2.968541937516619,
 3.1249706593905255,
 -2.4136436406710136,
 2.548105351482853)

In [12]:
loaded_dict[0][0].keys()

dict_keys(['Jz', 'θ1', 'Jx', 'θ2', 'θ3', 'θ4'])

In [13]:

circuits = []
circuit_realizations = 1000

for circuit_realization in range(circuit_realizations):
    gates = []
    for n in range(N):
        params = loaded_dict[circuit_realization][n]
        Jz = params['Jz']
        θ1 = params['θ1']
        Jx = params['Jx']
        θ2 = params['θ2']
        θ3 = params['θ3']
        θ4 = params['θ4']
        gates.append(fn.gate_xyz_disordered(θ1, θ2, Jx, Jx, Jz, θ3, θ4)) # h1, h2, Jx, Jy, Jz, h3, h4
    
    gates = np.array(gates)
    assert len(gates) == N, f"Expected {N} gates, got {gates.shape}"

    order = fn.gen_gates_order(N)    
    circuit = Circuit(N=N, gates=gates, order=order)
    circuits.append(circuit)
    
##############################################################################################

def compute_circuit(idx, circuit_real):
    circuit = circuits[circuit_real]
    state = initial_state(N, idx) # initial_state_test(theta)
    return circuit.run(masks_dict, state, T)

In [14]:
if globals().get('result') is None or globals(
    ).get('result').shape != (N, circuit_realizations, T + 1, N):
    result = np.zeros((N, circuit_realizations, T + 1, N), dtype=np.float64)


for idx, circuit_real in tqdm(product(
        range(circuit_realizations), #
        range(N), 
    ), total=N * circuit_realizations):
    result[idx, circuit_realization] = compute_circuit(idx, circuit_realization);

  1%|          | 117/16000 [01:18<2:58:38,  1.48it/s]


KeyboardInterrupt: 

In [ ]:
for i in range(N):
    for j in range(4):
        assert len(masks_dict[i][j]) == 2**N//4, f"mask {i} {j} has wrong shape: {len(masks_dict[i][j])} vs {2**N//4}"
print("All masks have the correct shape.")